# Generate Training Data

This notebook is similar to notebook 4 with the specific purpose to generate more training data for the model training in notebook 7. 

## Get queries

In [93]:
import pandas as pd
import numpy as np
import requests
import json
import itertools
from IPython.display import display, HTML, Image

In [94]:
DATA_DIR = '/Users/danielwrigley/work/Testing/git_repos/esci-data/shopping_queries_dataset/'

In [95]:
df_examples = pd.read_parquet(DATA_DIR + '/shopping_queries_dataset_examples.parquet')

In [96]:
# We only use English queries for now
df_queries_us = df_examples[df_examples['product_locale'] == 'us']

In [97]:
np.random.seed(10)

In [98]:
# Sample query sets

res = []

query_set = np.random.choice(df_queries_us["query"].unique(), 5000, replace=False)

df = pd.DataFrame({"query": query_set})
res.append(df)
df_query_set = pd.concat(res)
df_query_set.head(10)

,query
0,runtz
1,trooper bandana shoe
2,tcl a1x phone case straight talk
3,bose headphones replacement cord
4,uniball vision elite
5,definitely not paid enough for this
6,raid deep reach fogger
7,usb camera
8,reusable produce bags
9,latex dental dam


In [99]:
df_query_set.shape[0]

5000

In [100]:
# Select judgments
# Map esci_label to score
# create judgments per day in range
# create noise in score

label_num = {"E": 0, "S": 1, "C": 2, "I": 3}
label_score = [3, 2, 1, 0]
#label_score = [1, 0.1, 0.01, 0]

def label_to_score(label):
    return label_score[label_num[label]]

df_judge = df_examples[df_examples["query"].isin(set(df_query_set["query"].values))].copy()
df_judge["judgment"] = df_judge.esci_label.apply(lambda x: label_to_score(x))
df_judge["document"] = df_judge.product_id
df_judge = df_judge[["query", "document", "judgment", "query_id"]].reset_index(drop=True)
df_judge.head(3)

,query,document,judgment,query_id
0,#8 tags without string,B0751KS4ZW,0,44
1,#8 tags without string,B07541MJRV,2,44
2,#8 tags without string,B075WX3LFF,2,44


In [38]:
df_ratings = pd.merge(df_judge, df_query_set, on='query', how='left')
df_ratings.columns = ['query', 'docid', 'rating', 'idx']

In [102]:
df_ratings.head(3)

,query_string,product_id,rating,query_id
0,#8 tags without string,B0751KS4ZW,0,44
1,#8 tags without string,B07541MJRV,2,44
2,#8 tags without string,B075WX3LFF,2,44


## Query OpenSearch with the Hybrid Search Configurations

In [103]:
# Get model_id
# We are assuming that the installation has only one model. Change this if you have more models 
# and need to pick a specific one

url = "http://localhost:9200/_plugins/_ml/models/_search"

headers = {
    'Content-Type': 'application/json'
}

payload = {
  "query": {
    "match_all": {}
  },
  "size": 1
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

model_id = response.json()['hits']['hits'][0]['_source']['model_id']

## Create a DataFrame with all possible combinations of hybrid search configurations

In [104]:
# Define the possible values for each column
normalization_values = ['min_max', 'l2']
combination_values = ['arithmetic_mean', 'harmonic_mean', 'geometric_mean']
keyword_values = [round(i * 0.1, 1) for i in range(11)]

# Create all possible combinations of normalization, combination, and keyword
combinations = list(itertools.product(normalization_values, combination_values, keyword_values))

# Calculate the vector as 1.0 - keyword
data = [(norm, comb, kw, 1.0 - kw) for norm, comb, kw in combinations]

# Create DataFrame
df_hybrid_search_params = pd.DataFrame(data, columns=['normalization', 'combination', 'keyword', 'vector'])

# Create a column with a pipeline name made up of its components
df_hybrid_search_params['pipeline'] = df_hybrid_search_params.normalization.apply(str) + \
    df_hybrid_search_params.combination.apply(str) + df_hybrid_search_params.keyword.apply(str)

df_hybrid_search_params.head()

,normalization,combination,keyword,vector,pipeline
0,min_max,arithmetic_mean,0.0,1.0,min_maxarithmetic_mean0.0
1,min_max,arithmetic_mean,0.1,0.9,min_maxarithmetic_mean0.1
2,min_max,arithmetic_mean,0.2,0.8,min_maxarithmetic_mean0.2
3,min_max,arithmetic_mean,0.3,0.7,min_maxarithmetic_mean0.3
4,min_max,arithmetic_mean,0.4,0.6,min_maxarithmetic_mean0.4


In [105]:
def create_search_pipeline(df):
    for idx, row in df.iterrows():
        normalization = row['normalization']
        combination = row['combination']
        keyword = row['keyword']
        vector = round(row['vector'],1)
        pipeline = row['pipeline']

        payload = {
          "request_processors": [
            {
              "neural_query_enricher" : {
                "description": "Sets the default model ID at index and field levels",
                "default_model_id": model_id,
                "neural_field_default_id": {
                   "title_embeddings": model_id
                }
              }
            }
          ],
          "phase_results_processors": [
            {
              "normalization-processor": {
                "normalization": {
                  "technique": normalization
                },
                "combination": {
                  "technique": combination,
                  "parameters": {
                    "weights": [
                      keyword,
                      vector
                    ]
                  }
                }
              }
            }
          ]    
        }

        url = "http://localhost:9200/_search/pipeline/" + pipeline
        
        response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
        #mr.JSON(response.json(), level=1)
        #print(payload)

In [106]:
create_search_pipeline(df_hybrid_search_params)

In [ ]:
df_relevance = pd.DataFrame()
for config in df_hybrid_search_params.itertuples():
#for config in df_hybrid_search_params.head(1).itertuples():
    pipeline_name = config[5]
    print(pipeline_name)

    # Set pipeline 
     
    url = "http://localhost:9200/ecommerce/_search?search_pipeline=" + pipeline_name
    
    headers = {
        'Content-Type': 'application/json'
    }
    # iterate over all query strings and send a hybrid search query to OpenSearch with the set pipeline
    for query in df_query_set.itertuples():
    
        payload = {
          "_source": {
            "excludes": [
              "title_embedding"
            ]
          },
          "query": {
            "hybrid": {
              "queries": [
                {
                  "multi_match" : {
                      "type":       "best_fields",
                      "fields":     [
                        "product_id^100",
                        "product_bullet_point^3",
                        "product_color^2",
                        "product_brand^5",
                        "product_description",
                        "product_title^10"
                      ],
                      "operator":   "and",
                      "query":      query[1]
                    }
                },
                {
                  "neural": {
                    "title_embedding": {
                      "query_text": query[1],
                      "k": 100
                    }
                  }
                }
              ]
            }
          },
          "size": 10
        }
    
        response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
        # store results per pipeline_id
        position = 0
        for hit in response['hits']['hits']:
            # create a new row for the DataFrame and append it
            row = { 'query_id' : str(query[0]), 'query_string': query[1], 'product_id' : hit["_id"], 'position' : str(position), 'relevance' : hit["_score"], 'run': pipeline_name }
    
            new_row_df = pd.DataFrame([row])
            df_relevance = pd.concat([df_relevance, new_row_df], ignore_index=True)
            #print("%(id)s %(title)s: %(name)s" % hit["_source"])
            position += 1
    
    # work with two for loops:
    # 1) one to iterate over the list of queries and have a query id instead of a query
    # 2) another one to iterate over the result sets to have the position of the result in the result set 
    
    # DataFrame with columns:
    # query_id: the id of the query as the trec_eval tool needs a numeric id rather than a query string as an identifier
    # product_id: the id of the product in the hit list
    # position: the position of the product in the result set
    # relevance: relevance as given by the search engine
    # run: the name of the query pipeline

min_maxarithmetic_mean0.0
min_maxarithmetic_mean0.1
min_maxarithmetic_mean0.2
min_maxarithmetic_mean0.3
min_maxarithmetic_mean0.4
min_maxarithmetic_mean0.5
min_maxarithmetic_mean0.6
min_maxarithmetic_mean0.7
min_maxarithmetic_mean0.8
min_maxarithmetic_mean0.9
min_maxarithmetic_mean1.0
min_maxharmonic_mean0.0
min_maxharmonic_mean0.1
min_maxharmonic_mean0.2
min_maxharmonic_mean0.3
min_maxharmonic_mean0.4
min_maxharmonic_mean0.5
min_maxharmonic_mean0.6
min_maxharmonic_mean0.7
min_maxharmonic_mean0.8
min_maxharmonic_mean0.9
min_maxharmonic_mean1.0
min_maxgeometric_mean0.0
min_maxgeometric_mean0.1
min_maxgeometric_mean0.2
min_maxgeometric_mean0.3
min_maxgeometric_mean0.4
min_maxgeometric_mean0.5
min_maxgeometric_mean0.6
min_maxgeometric_mean0.7
min_maxgeometric_mean0.8
min_maxgeometric_mean0.9
min_maxgeometric_mean1.0
l2arithmetic_mean0.0
l2arithmetic_mean0.1
l2arithmetic_mean0.2
l2arithmetic_mean0.3
l2arithmetic_mean0.4
l2arithmetic_mean0.5
l2arithmetic_mean0.6
l2arithmetic_mean0.7
l2arith

In [ ]:
df_relevance.head(3)

There are 10 results per query, so there are _number of queries_ * 10 rows per pipeline in the resulting DataFrame

In [ ]:
df_relevance.shape[0]

# Calculate Metrics per Pipeline

In [ ]:
# Rename the columns to match the column names in df_relevance
df_ratings.columns = ['query_string', 'product_id', 'rating', 'query_id']
df_ratings.head(3)

In [ ]:
# Remove duplicates from the ratings DataFrame
df_unique_ratings = df_ratings.drop_duplicates(subset=['product_id', 'query_string'])

In [ ]:
# Merge results on query_string and product_id so that the resulting DataFrame has the ratings together with the search results
# Validations helps us make sure that we have only one rating for each query-doc pair. We have identical query-doc pairs per
# search pipeline but we can only have one rating for these.
df_merged = df_relevance.merge(df_unique_ratings, on=['query_string', 'product_id'], how='left', validate='many_to_one')
df_merged = df_merged.drop(columns=['query_id_y'])

df_merged.head(3)
df_merged = df_merged.rename(columns={"query_id_x": "query_id"})

In [ ]:
# Have a look at one query to see that the merge worked
df_merged[df_merged['query_string'] == '(fiction without frontiers)'].head(5)

In [ ]:
# Count the rows without ratings - the higher the count is the less reliable the results will be
nan_count_rating = df_merged['rating'].isna().sum()
print(f"There are {df_merged.shape[0]} rows and {nan_count_rating} do not contain a rating")

## Metrics

In [ ]:
def dcg_at_10(df, k=10, position=None, **kwargs):
    # Sort by position and take the top k results
    # if position is given, the caller has already sorted the dataframe
    # an additional position argument is used to avoid excessive dataframe copying
    if position is None:
        df = df.sort_values('position')
        position = df["position"]
    if k:
        df = df.head(k)
        position = position[:10]

    # Apply DCG formula
    dcg = np.sum(2.0 ** (df['rating'] - 1) / np.log2(position + 2))
    
    return dcg


def ndcg_at_10(df, reference=None):
    # Calculate DCG@10
    dcg = dcg_at_10(df)
    
    # Use reference judgments, because best ratings may be outside of search results
    assert reference is not None

    # Reset the positional information - otherwise it uses the original positions and the 
    # changed sorting would have no effect
    ideal_top_10 = reference.sort_values("rating", ascending=False)
    position = np.arange(ideal_top_10.shape[0])
    
    # Calculate iDCG (ideal DCG)
    idcg = dcg_at_10(ideal_top_10, position=position, k=None)
    
    # Handle cases where iDCG is 0
    if idcg == 0:
        return 0
    
    # Normalize DCG
    ndcg = dcg / idcg
    
    return ndcg

def dg_at_10(df, k=10, **kwargs):
    """Failed attempt at performing a different aggregation for a DCG-like metric"""
    # Sort by position and take the top k results
    if k:
        df = df.sort_values('position').head(k)
    
    # Calculate DG for each result and average non-NaNs
    dgs = df['rating'] / np.log2(df['position'] + 2)
    dgs = dgs[~np.isnan(dgs)]

    if dgs.shape[0] == 0:
        return 0 # penalize queries with no judgements (alternative, not include in averaging through the use of weights)
    else:
        return dgs.mean()


def precision_at_k(df, k=10, **kwargs):
    # Sort by position and take the top k results
    top_k = df.sort_values('position').head(k)
    
    # Calculate the number of relevant results (assuming relevance > 1 is relevant)
    relevant_count = np.sum(top_k['rating'] > 1)
    
    # Calculate precision
    precision = relevant_count / k
    
    return precision

def ratio_of_ratings(df, k=10, **kwargs):
    top_k = df.sort_values('position').head(k)
    num_of_ratings = top_k[~top_k['rating'].isna()].shape[0]
    num_of_shown_results = top_k.shape[0]
    if num_of_shown_results == 0:
        return 0
    else:
        return num_of_ratings/num_of_shown_results

def condensed(metric_fun):
    """Turn metric function into calculation of condensed results"""
    # TODO: add reference
    def fun(df, **kwargs):
        df = df[~np.isnan(df.rating)]
        df = df.sort_values('position')
        return metric_fun(df, position=np.arange(df.shape[0]), **kwargs)
    return fun

metrics = [
    ("dcg", dcg_at_10, None),
    ("ndcg", ndcg_at_10, None),
    #("dg", dg_at_10, None),
    #("dcg_dense", lambda x, **kwargs: dcg_at_10(x, dense=True, **kwargs), None),
    ("prec@10", lambda x, **kwargs: precision_at_k(x, k=10, **kwargs), None),
    ("ratio_of_ratings", ratio_of_ratings, None)
]

## Calculate Metrics

Iterate over the queries in the query set, calculate the three metrics dcg@10, ndcg@10 and precision@10 and store the results for every query in a DataFrame

In [ ]:
reference = {query: df for query, df in df_ratings.groupby("query_string")}

df_metrics = []
for m_name, m_function, ref_search in metrics:
    for (query_string, run), df_gr in df_merged.groupby(["query_string", "run"]):
        metric = m_function(df_gr, reference=reference[query_string])
        df_metrics.append(pd.DataFrame({
            "query": [query_string],
            "pipeline": [run],
            "metric": [m_name],
            "value": [metric],
        }))
df_metrics = pd.concat(df_metrics)

In [ ]:
df_metrics.head(3)

In [ ]:
df_metrics.to_csv('metrics_5000_queries.csv', index=False)

## Calculate Metrics per Pipeline by Averaging the Query Metrics

In [91]:
df_metrics_per_pipeline = df_metrics.pivot_table(index="pipeline", columns="metric", values="value", aggfunc=lambda x: x.mean().round(2))
df_metrics_per_pipeline = df_metrics_per_pipeline.reset_index()

### Top five Pipelines by NDCG@10 Descending

In [92]:
df_metrics_per_pipeline.sort_values(by='ndcg', ascending=False).head(5)

metric,pipeline,dcg,dcg_dense,dg,ndcg,prec@10,ratio_of_ratings
0,min_maxarithmetic_mean0.0,4.57,4.57,1.0,0.21,0.24,0.25


### Top five Pipelines by DCG@10 Descending

In [136]:
df_metrics_per_pipeline.sort_values(by='dcg', ascending=False).head(5)

metric,pipeline,dcg,dcg_dense,dg,ndcg,prec@10,ratio_of_ratings
4,l2arithmetic_mean0.4,6.74,6.74,1.11,0.31,0.35,0.09
39,min_maxarithmetic_mean0.6,6.72,6.72,1.13,0.30,0.35,0.09
3,l2arithmetic_mean0.3,6.72,6.72,1.12,0.30,0.35,0.08
5,l2arithmetic_mean0.5,6.72,6.72,1.13,0.30,0.35,0.09
6,l2arithmetic_mean0.6,6.71,6.71,1.13,0.30,0.35,0.09


### Top five Pipelines by Precision@10 Descending

In [137]:
df_metrics_per_pipeline.sort_values(by='prec@10', ascending=False).head(5)

metric,pipeline,dcg,dcg_dense,dg,ndcg,prec@10,ratio_of_ratings
41,min_maxarithmetic_mean0.8,6.63,6.63,1.12,0.30,0.35,0.09
39,min_maxarithmetic_mean0.6,6.72,6.72,1.13,0.30,0.35,0.09
3,l2arithmetic_mean0.3,6.72,6.72,1.12,0.30,0.35,0.08
4,l2arithmetic_mean0.4,6.74,6.74,1.11,0.31,0.35,0.09
5,l2arithmetic_mean0.5,6.72,6.72,1.13,0.30,0.35,0.09


In [138]:
df_merged.to_csv('results_and_ratings.csv')

In [139]:
# Use query (fiction without frontiers) for random query set, and 3 ring binder for most judged query set

query = '(fiction without frontiers)'
#query = '3 ring binder'

df_merged[(df_merged['query_string'] == query) & (df_merged['run'] == 'min_maxarithmetic_mean0.0')]

,query_id,query_string,product_id,position,relevance,run,rating
100,1,(fiction without frontiers),B07GJVWWWR,0,1.000000,min_maxarithmetic_mean0.0,3.0
101,1,(fiction without frontiers),B08C5MQFCY,1,0.935822,min_maxarithmetic_mean0.0,3.0
102,1,(fiction without frontiers),B07XPDNFL2,2,0.920334,min_maxarithmetic_mean0.0,NaN
103,1,(fiction without frontiers),B07PGQF8K2,3,0.913474,min_maxarithmetic_mean0.0,3.0
104,1,(fiction without frontiers),1787583325,4,0.812228,min_maxarithmetic_mean0.0,NaN
...,...,...,...,...,...,...,...
195,1,(fiction without frontiers),0062917455,95,0.006625,min_maxarithmetic_mean0.0,NaN
196,1,(fiction without frontiers),B07SPG69Z2,96,0.005574,min_maxarithmetic_mean0.0,NaN
197,1,(fiction without frontiers),B0015DWNR4,97,0.004980,min_maxarithmetic_mean0.0,NaN
198,1,(fiction without frontiers),0449148378,98,0.001000,min_maxarithmetic_mean0.0,NaN


In [140]:
df_metrics[(df_metrics['query'] == query) & (df_metrics['pipeline'] == 'min_maxarithmetic_mean0.0')]

,query,pipeline,metric,value
0,(fiction without frontiers),min_maxarithmetic_mean0.0,dcg,9.579759
0,(fiction without frontiers),min_maxarithmetic_mean0.0,ndcg,0.442371
0,(fiction without frontiers),min_maxarithmetic_mean0.0,dg,1.796205
0,(fiction without frontiers),min_maxarithmetic_mean0.0,dcg_dense,9.579759
0,(fiction without frontiers),min_maxarithmetic_mean0.0,prec@10,0.400000
0,(fiction without frontiers),min_maxarithmetic_mean0.0,ratio_of_ratings,0.120000


### Use the following code to double check that the results and the judgements match

Requires knowing a query from the current query set, e.g. airpods for the most judged query set, (fiction wihtout frontiers) for the random query set

In [141]:
DATA_DIR = '/Users/danielwrigley/work/Testing/git_repos/esci-data/shopping_queries_dataset/'

In [142]:
df_examples = pd.read_parquet(DATA_DIR + '/shopping_queries_dataset_examples.parquet')

In [143]:
df_examples[(df_examples['query'] == query) & (df_examples['product_locale'] == 'us')]

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
5172,5172,(fiction without frontiers),170,B08C5KY8V8,us,I,0,1,train
5173,5173,(fiction without frontiers),170,B08NLC5VJC,us,S,0,1,train
5174,5174,(fiction without frontiers),170,B08NKCZM5Y,us,E,0,1,train
5175,5175,(fiction without frontiers),170,B08LDTWY9L,us,I,0,1,train
5176,5176,(fiction without frontiers),170,B08CVS88GV,us,S,0,1,train
5177,5177,(fiction without frontiers),170,B08C5N9F3G,us,E,0,1,train
5178,5178,(fiction without frontiers),170,B08C5MQFCY,us,E,0,1,train
5179,5179,(fiction without frontiers),170,B088JLGCWB,us,E,0,1,train
5180,5180,(fiction without frontiers),170,B086689HYL,us,E,0,1,train
5181,5181,(fiction without frontiers),170,B07SNF45NV,us,E,0,1,train


In [144]:
query_id = str(df_query_idx[df_query_idx['query'] == query]['idx'][1])
print(query_id)

1


In [145]:
df_ratings[df_ratings['query_id'] == query_id]

,query_string,product_id,rating,query_id
15,(fiction without frontiers),B08C5KY8V8,0,1
16,(fiction without frontiers),B08NLC5VJC,2,1
17,(fiction without frontiers),B08NKCZM5Y,3,1
18,(fiction without frontiers),B08LDTWY9L,0,1
19,(fiction without frontiers),B08CVS88GV,2,1
20,(fiction without frontiers),B08C5N9F3G,3,1
21,(fiction without frontiers),B08C5MQFCY,3,1
22,(fiction without frontiers),B088JLGCWB,3,1
23,(fiction without frontiers),B086689HYL,3,1
24,(fiction without frontiers),B07SNF45NV,3,1


In [146]:
df_merged[df_merged['query_string'] == query]['product_id']

100        B07GJVWWWR
101        B08C5MQFCY
102        B07XPDNFL2
103        B07PGQF8K2
104        1787583325
              ...    
1430195    0312367465
1430196    B074ZX8N2Q
1430197    1525823566
1430198    B000093OU8
1430199    B00HJCAGR4
Name: product_id, Length: 6600, dtype: object

In [147]:
df_examples[(df_examples['query'] == query) & (df_examples['product_locale'] == 'us') & df_examples['product_id'].isin(df_merged[df_merged['query_string'] == query]['product_id'])]

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
5172,5172,(fiction without frontiers),170,B08C5KY8V8,us,I,0,1,train
5177,5177,(fiction without frontiers),170,B08C5N9F3G,us,E,0,1,train
5178,5178,(fiction without frontiers),170,B08C5MQFCY,us,E,0,1,train
5179,5179,(fiction without frontiers),170,B088JLGCWB,us,E,0,1,train
5180,5180,(fiction without frontiers),170,B086689HYL,us,E,0,1,train
5181,5181,(fiction without frontiers),170,B07SNF45NV,us,E,0,1,train
5182,5182,(fiction without frontiers),170,B07QJX9NCV,us,E,0,1,train
5183,5183,(fiction without frontiers),170,B07PGQF8K2,us,E,0,1,train
5184,5184,(fiction without frontiers),170,B07PDMZWY7,us,E,0,1,train
5185,5185,(fiction without frontiers),170,B07L9JSMT8,us,E,0,1,train
